<a href="https://colab.research.google.com/github/binniehoang/project/blob/master/project472revised2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%writefile project.c
#include <stdio.h>
#include <string.h>
#include <sys/ipc.h>
#include <sys/shm.h>
#include <unistd.h>
#include <ctype.h>
#include <pthread.h>
#include <stdlib.h>


#define MAX_SEGMENTS 10
#define MAX_THREADS 10
#define MAX_PROCESSES 10
#define SHARED_MEMORY_KEY 1234
#define MESSAGE_SIZE 256
#define SEGMENT_SIZE 1000

typedef struct {
    int id;
    char status[20];
} Thread;

typedef struct {
    int id;
    char status[20];
    Thread threads[MAX_THREADS];
    int thread_count;
} Process;

typedef struct {
    Process processes[MAX_PROCESSES];
    int process_count;
} TaskManager;

typedef struct {
    char* text;
    int start;
    int end;
} ThreadData;

// creating interprocess communication mechanisms
typedef struct {
    pthread_mutex_t mutex;
    char message[MESSAGE_SIZE];
    int flag;
    int terminate;
} SharedData;


// Parallel file processing
typedef struct {
    char segment[SEGMENT_SIZE];
} Segment;

typedef struct {
    char character;
    int count;
} CharacterCount;

typedef struct {
    Segment segments[MAX_SEGMENTS];
    int segment_count;
    CharacterCount counts[256]; // Assuming ASCII characters
} TextProcessor;


void display_menu(){
    printf("1. Create Process\n");
    printf("2. Create Thread\n");
    printf("3. Display Processes\n");
    printf("4. Kill Process\n");
    printf("5. Suspend Process\n");
    printf("6. Resume Process\n");
    printf("7. Kill Thread\n");
    printf("8. Suspend Thread\n");
    printf("9. Resume Thread\n");
    printf("10. Send Message (IPC)\n");
    printf("11. Evaluate IPC performance\n");
    printf("12. Parallel Text Processing\n");
    printf("13. Exit\n");

}



void* process_text(void* arg) {
    ThreadData* data = (ThreadData*) arg;
    for (int i = data->start; i < data->end; i++) {
        data->text[i] = toupper(data->text[i]);
    }
    return NULL;
}
// Function to kill a specific process
void kill_process(TaskManager* tm, int process_id) {
    for (int i = 0; i < tm->process_count; i++) {
        if (tm->processes[i].id == process_id) {
            // Logic to terminate the process
            printf("Process with ID %d terminated.\n", process_id);
            // Optional: Update process status to "Terminated"
            strcpy(tm->processes[i].status, "Terminated");
            return;
        }
    }
    printf("Process with ID %d not found.\n", process_id);
}

// Function to suspend a specific process
void suspend_process(TaskManager* tm, int process_id) {
    for (int i = 0; i < tm->process_count; i++) {
        if (tm->processes[i].id == process_id) {
            // Logic to suspend the process
            printf("Process with ID %d suspended.\n", process_id);
            // Optional: Update process status to "Suspended"
            strcpy(tm->processes[i].status, "Suspended");
            return;
        }
    }
    printf("Process with ID %d not found.\n", process_id);
}

// Function to resume a specific process
void resume_process(TaskManager* tm, int process_id) {
    for (int i = 0; i < tm->process_count; i++) {
        if (tm->processes[i].id == process_id) {
            // Logic to resume the process
            printf("Process with ID %d resumed.\n", process_id);
            // Optional: Update process status to "Running"
            strcpy(tm->processes[i].status, "Running");
            return;
        }
    }
    printf("Process with ID %d not found.\n", process_id);
}

// Function to kill a specific thread within a process
void kill_thread(TaskManager* tm, int process_id, int thread_id) {
    for (int i = 0; i < tm->process_count; i++) {
        if (tm->processes[i].id == process_id) {
            for (int j = 0; j < tm->processes[i].thread_count; j++) {
                if (tm->processes[i].threads[j].id == thread_id) {
                    // Logic to terminate the thread
                    printf("Thread with ID %d in process %d terminated.\n", thread_id, process_id);
                    // Optional: Update thread status to "Terminated"
                    strcpy(tm->processes[i].threads[j].status, "Terminated");
                    return;
                }
            }
            printf("Thread with ID %d not found in process %d.\n", thread_id, process_id);
            return;
        }
    }
    printf("Process with ID %d not found.\n", process_id);
}

// function to create process
void create_process(TaskManager* tm, int id, char* status) {
    Process p;
    p.id = id;
    strcpy(p.status, status);
    p.thread_count = 0;
    tm->processes[tm->process_count++] = p;
}

void create_thread(TaskManager* tm, int process_id, int thread_id, char* status) {
    for (int i = 0; i < tm->process_count; i++) {
        if (tm->processes[i].id == process_id) {
            if (tm->processes[i].thread_count < MAX_THREADS) {
                Thread t;
                t.id = thread_id;
                strcpy(t.status, status);
                tm->processes[i].threads[tm->processes[i].thread_count++] = t;
            } else {
                printf("Maximum thread count reached for the process.\n");
            }
            return;
        }
    }
    printf("Process with ID %d not found.\n", process_id);
}

//function to display process
void display_processes(TaskManager* tm) {

    printf("Processes:\n");
    for (int i = 0; i < tm->process_count; i++) {
        printf("Process ID: %d, Status: %s\n", tm->processes[i].id, tm->processes[i].status);
        printf("Threads:\n");
        for (int j = 0; j < tm->processes[i].thread_count; j++) {
            printf("Thread ID: %d, Status: %s\n", tm->processes[i].threads[j].id, tm->processes[i].threads[j].status);
        }
    }
}
//function to suspend thread
void suspend_thread(TaskManager* tm, int process_id, int thread_id) {
    for (int i = 0; i < tm->process_count; i++) {
        if (tm->processes[i].id == process_id) {
            for (int j = 0; j < tm->processes[i].thread_count; j++) {
                if (tm->processes[i].threads[j].id == thread_id) {
                    strcpy(tm->processes[i].threads[j].status, "Suspended");
                    printf("Thread with ID %d in process %d suspended.\n", thread_id, process_id);
                    return;
                }
            }
            printf("Thread with ID %d not found in process %d.\n", thread_id, process_id);
            return;
        }
    }
    printf("Process with ID %d not found.\n", process_id);
}
// function to resume thread
void resume_thread(TaskManager* tm, int process_id, int thread_id) {
    for (int i = 0; i < tm->process_count; i++) {
        if (tm->processes[i].id == process_id) {
            for (int j = 0; j < tm->processes[i].thread_count; j++) {
                if (tm->processes[i].threads[j].id == thread_id) {
                    strcpy(tm->processes[i].threads[j].status, "Running");
                    printf("Thread with ID %d in process %d resumed.\n", thread_id, process_id);
                    return;
                }
            }
            printf("Thread with ID %d not found in process %d.\n", thread_id, process_id);
            return;
        }
    }
    printf("Process with ID %d not found.\n", process_id);
}

//Feature 2
//Interprocess communication mechanisms
// Shared memory and message passing for processes
void* process_ipc(void* arg) {
    SharedData* shared_data = (SharedData*)arg;
    while (1) {
        pthread_mutex_lock(&shared_data->mutex);
        if (shared_data->flag) {
            printf("Process received message: %s\n", shared_data->message);
            shared_data->flag = 0;
        }
        pthread_mutex_unlock(&shared_data->mutex);
        sleep(1); // Simulating processing time
    }
    return NULL;
}

// Shared memory and message passing for threads
void* thread_ipc(void* arg) {
    SharedData* shared_data = (SharedData*)arg;
    while (1) {
        pthread_mutex_lock(&shared_data->mutex);
        if (shared_data->flag) {
            printf("Thread received message: ");
            if(strlen(shared_data->message) > 50) {
                printf("%.50s...\n", shared_data->message); // Display only first 50 characters
            } else {
                printf("%s\n", shared_data->message);
            }
            shared_data->flag = 0; // Reset flag after processing the message
        }
        pthread_mutex_unlock(&shared_data->mutex);
        sleep(1); // Simulating processing time
    }
    return NULL;
}
// Function to handle sending messages between processes or threads
void send_message_ipc(SharedData* shared_data) {
    printf("Enter message to send: ");
    char message[MESSAGE_SIZE];
    scanf("%s", message);

    pthread_mutex_lock(&shared_data->mutex);
    strcpy(shared_data->message, message);
    shared_data->flag = 1;
    pthread_mutex_unlock(&shared_data->mutex);

    printf("Message sent successfully.\n");
}
// Evaluate and compare performance of IPC mechanisms
void evaluate_ipc_performance(SharedData* shared_data) {
    clock_t start, end;
    double cpu_time_used;

    // Sending short messages
    start = clock();
    strcpy(shared_data->message, "Short message");
    shared_data->flag = 1;
    end = clock();
    cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
    printf("Time taken to send a short message: %f seconds\n", cpu_time_used);

    // Sending long messages
    start = clock();
    char long_message[MESSAGE_SIZE];
    memset(long_message, 'A', MESSAGE_SIZE - 1);
    long_message[MESSAGE_SIZE - 1] = '\0'; // Null-terminate the string
    strcpy(shared_data->message, long_message);
    shared_data->flag = 1;
    end = clock();
    cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
    printf("Time taken to send a long message: %f seconds\n", cpu_time_used);
}

// 3rd feature parallel text file processing
void load_file(TextProcessor* tp) {
    FILE* file = fopen("project.c.txt", "r");
    if (file == NULL) {
        printf("Error opening file.\n");
        exit(1);
    }

    char buffer[SEGMENT_SIZE];
    while (fgets(buffer, sizeof(buffer), file) != NULL) {
        strcpy(tp->segments[tp->segment_count].segment, buffer);
        tp->segment_count++;
    }

    fclose(file);
}

void* process_segment(void* arg) {
    Segment* segment = (Segment*)arg;
    for (int i = 0; i < strlen(segment->segment); i++) {
        if (isalpha(segment->segment[i])) {
            segment->segment[i] = toupper(segment->segment[i]);
        }
    }
    return NULL;
}

void* count_characters(void* arg) {
    TextProcessor* tp = (TextProcessor*)arg;
    for (int i = 0; i < tp->segment_count; i++) {
        for (int j = 0; j < strlen(tp->segments[i].segment); j++) {
            if (isalpha(tp->segments[i].segment[j])) {
                tp->counts[(int)tp->segments[i].segment[j]].character = tp->segments[i].segment[j];
                tp->counts[(int)tp->segments[i].segment[j]].count++;
            }
        }
    }
    return NULL;
}

void parallel_text_processing() {
    TextProcessor tp;
    tp.segment_count = 0;

    // Load file into segments
    load_file(&tp);

    // Create threads for processing segments
    pthread_t process_threads[MAX_THREADS];
    for (int i = 0; i < tp.segment_count; i++) {
        pthread_create(&process_threads[i], NULL, process_segment, (void*)&tp.segments[i]);
    }

    // Wait for processing threads to complete
    for (int i = 0; i < tp.segment_count; i++) {
        pthread_join(process_threads[i], NULL);
    }

    // Create thread for counting characters
    pthread_t count_thread;
    pthread_create(&count_thread, NULL, count_characters, (void*)&tp);
    pthread_join(count_thread, NULL);

    // Display character counts
    printf("Character Counts:\n");
    for (int i = 0; i < 256; i++) {
        if (isalpha(tp.counts[i].character)) {
            printf("%c: %d\n", tp.counts[i].character, tp.counts[i].count);
        }
    }
}


int main() {
    SharedData shared_data;
    pthread_mutex_init(&shared_data.mutex, NULL);
    shared_data.flag = 0;


    TaskManager tm;
    tm.process_count = 0;
    int choice;
    do {
        display_menu();
        printf("Enter your choice: ");
        scanf("%d", &choice);

        switch (choice) {
            case 1:
                // Code to create a process
                printf("Enter process ID: ");
                int process_id;
                scanf("%d", &process_id);
                printf("Enter process status: ");
                char process_status[20];
                scanf("%s", process_status);
                create_process(&tm, process_id, process_status);
                break;
            case 2:
                // Code to create a thread
                printf("Enter process ID for the new thread: ");
                scanf("%d", &process_id);
                printf("Enter thread ID: ");
                int thread_id;
                scanf("%d", &thread_id);
                printf("Enter thread status: ");
                char thread_status[20];
                scanf("%s", thread_status);
                create_thread(&tm, process_id, thread_id, thread_status);
                break;
            case 3:
                display_processes(&tm);
                break;

          case 4:
            // Code to kill a process
            printf("Enter process ID to kill: ");
            scanf("%d", &process_id);
            kill_process(&tm, process_id);
            break;
          case 5:
            // Code to suspend a process
            printf("Enter process ID to suspend: ");
            scanf("%d", &process_id);
            suspend_process(&tm, process_id);
            break;
          case 6:
            // Code to resume a process
            printf("Enter process ID to resume: ");
            scanf("%d", &process_id);
            resume_process(&tm, process_id);
            break;
          case 7:
            // Code to kill a thread
            printf("Enter process ID: ");
            scanf("%d", &process_id);
            printf("Enter thread ID to kill: ");
            scanf("%d", &thread_id);
            kill_thread(&tm, process_id, thread_id);
            break;
          case 8:
            // Code to suspend a thread
            printf("Enter process ID: ");
            scanf("%d", &process_id);
            printf("Enter thread ID to suspend: ");
            scanf("%d", &thread_id);
            suspend_thread(&tm, process_id, thread_id);
                break;
          case 9:
            // Code to resume a thread
            printf("Enter process ID: ");
            scanf("%d", &process_id);
            printf("Enter thread ID to resume: ");
            scanf("%d", &thread_id);
            resume_thread(&tm, process_id, thread_id);
            break;
          case 10:
            send_message_ipc(&shared_data);
            break;
          case 11:
          evaluate_ipc_performance(&shared_data);
          break;
          case 13:
            printf("Exiting...\n");
            break;
            default:
                printf("Invalid choice. Please try again.\n");
        }
    } while (choice != 13);

    parallel_text_processing();



    // Cleanup
    pthread_mutex_destroy(&shared_data.mutex);


    return 0;
}




Overwriting project.c


In [14]:
%%shell
gcc project.c -o project
./project

1. Create Process
2. Create Thread
3. Display Processes
4. Kill Process
5. Suspend Process
6. Resume Process
7. Kill Thread
8. Suspend Thread
9. Resume Thread
10. Send Message (IPC)
11. Evaluate IPC performance
12. Parallel Text Processing
13. Exit
Enter your choice: 12
Invalid choice. Please try again.
1. Create Process
2. Create Thread
3. Display Processes
4. Kill Process
5. Suspend Process
6. Resume Process
7. Kill Thread
8. Suspend Thread
9. Resume Thread
10. Send Message (IPC)
11. Evaluate IPC performance
12. Parallel Text Processing
13. Exit
Enter your choice: 13
Exiting...
Error opening file.


CalledProcessError: Command 'gcc project.c -o project
./project
' returned non-zero exit status 1.